# Оконные функции

###### Задание 1 

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***), для этого необходимо выполнить коррелированный подзапрос, обратившись к внешней таблице ***episodes***;

— номер сезона;

— номер эпизода в сезоне;

— сквозной номер (назовите поле ***rn***);

Все названия хранятся в поле ***original_title*** таблицы ***titles***.

Партицирование сделайте по ***id*** названия сериала.

Результат отсортируйте по возрастанию ***serial_name***, ***episode_name***, ***season***, ***episode***, ***rn***.

``` PLSQL
with cte as (SELECT (SELECT original_title
                                  FROM   titles as t
                                  WHERE  t.id = e.parent_id) as serial_name, (SELECT original_title
                                                         FROM   titles as t2
                                                         WHERE  t2.id = e.title_id) as episode_name, season, episode, parent_id
             FROM   episodes as e
             WHERE  (SELECT original_title
                     FROM   titles as t
                     WHERE  t.id = e.parent_id) is not null)
SELECT serial_name,
       episode_name,
       season,
       episode,
       row_number() OVER (PARTITION BY parent_id
                          ORDER BY season, episode) as rn
FROM   cte
ORDER BY 1, 2, 3, 4, 5
```
###### Задание 2 

Напишите SQL-запрос, который выведет количество умерших людей из таблицы ***persons*** для каждого года, начиная с 2000-го, накопленным итогом.

Например, для 2012 года будет указано количество людей, которые ушли из жизни с 2000 по 2012 год.

Поля к выводу: год (***year_of_death***), кумулятивная сумма на этот год (***cume_cnt***). Отсортируйте результат по их возрастанию.

``` PLSQL
SELECT year_of_death,
       sum(count(*)) OVER (ORDER BY year_of_death) as cume_cnt
FROM   persons
GROUP BY year_of_death having year_of_death > 1999
ORDER BY 1, 2
```

###### Задание 3

Напишите SQL-запрос, который выведет:

— ***year_of_start*** из таблицы ***titles***;

— количество произведений по годам накопленным итогом, набравших максимальный средний рейтинг, равный 10.

Для выполнения запроса используйте таблицы ***titles*** и ***ratings***. Поля к выводу: год (***year_of_start***), кумулятивная сумма (***cum_cnt***). Отсортируйте результат по их возрастанию.

``` PLSQL
with cte as (SELECT t.year_of_start,
                    count(1) as cnt
             FROM   titles as t join ratings as r
                     ON t.id = r.title_id
             WHERE  avg_rating = 10
             GROUP BY year_of_start)
SELECT year_of_start,
       sum(cnt) OVER (ORDER BY year_of_start) as cume_cnt
FROM   cte
ORDER BY 1, 2;
```


###### Задание 4

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— максимальный рейтинг эпизодов до этой серии включительно (назовите поле ***max_rating***);

Серии с пустыми значениями сезона и эпизода показывать не нужно.

Результат отсортируйте по всем колонкам в том порядке, в котором они расположены выше (сортировка в порядке возрастания).

``` PLSQL
with cte as (SELECT e.parent_id as id,
                    e.season as season,
                    e.episode as episode,
                    (SELECT original_title
              FROM   titles as t
              WHERE  t.id = e.parent_id) as serial_name, (SELECT original_title
                                                         FROM   titles as t
                                                         WHERE  t.id = e.title_id) as episode_name, (SELECT avg_rating
                                                         FROM   ratings as r
                                                         WHERE  r.title_id = e.title_id) as avg_rating
             FROM   episodes as e
             WHERE  (SELECT original_title
                     FROM   titles as t
                     WHERE  t.id = e.parent_id) is not null
                and (SELECT original_title
                  FROM   titles as t
                  WHERE  t.id = e.title_id) is not null
                and e.season is not null
                and e.episode is not null
                and (SELECT avg_rating
                  FROM   ratings as r
                  WHERE  r.title_id = e.title_id) is not null)
SELECT serial_name,
       episode_name,
       season,
       episode,
       max(avg_rating) OVER (PARTITION BY id
                             ORDER BY season, episode) as max_rating
FROM   cte
ORDER BY 1, 2, 3, 4, 5
```

###### Задание 5

Напишите SQL-запрос, который выведет:

— название сериала  (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— количество голосов, отданных за сериал до текущего эпизода включительно (назовите поле ***vote_cnt***);

Результат отсортируйте по возрастанию ***serial_name***, ***episode_name***, ***season***, ***episode***, ***vote_cnt***.

``` PLSQL
with cte as (SELECT t.original_title as serial_name,
                    (SELECT t.original_title
              FROM   titles as t
              WHERE  t.id = e.title_id) as episode_name, e.parent_id, e.season, e.episode, r.vote_cnt
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id join titles as t
                     ON t.id = e.parent_id)
SELECT serial_name,
       episode_name,
       season,
       episode,
       sum(vote_cnt) OVER (PARTITION BY parent_id
                           ORDER BY season, episode range between unbounded preceding and current row) as vote_cnt
FROM   cte
ORDER BY 1, 2, 3, 4, 5
```

###### Задание 6

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— расчетное поле: если предыдущий эпизод получил оценку хуже — ***better***, если лучше — ***worse***, если такую же — ***the same***. Для первого эпизода значение принимается ***NULL***. Назовите поле ***mark***.

Результат отсортируйте по возрастанию ***serial_name***, ***season***, ***episode***, ***episode_name***.

``` PLSQL
with cte as (SELECT t.original_title as serial_name,
                    (SELECT t.original_title
              FROM   titles as t
              WHERE  t.id = e.title_id) as episode_name, e.parent_id, e.season, e.episode, r.avg_rating
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id join titles as t
                     ON t.id = e.parent_id)
SELECT serial_name,
       episode_name,
       season,
       episode,
       case when avg_rating > (lag (avg_rating) OVER (PARTITION BY parent_id
                                                      ORDER BY season, episode)) then 'better'
       when avg_rating < (lag (avg_rating) OVER (PARTITION BY parent_id
                                                      ORDER BY season, episode)) then 'worse'
       when avg_rating = (lag (avg_rating) OVER (PARTITION BY parent_id
                                                      ORDER BY season, episode)) then 'the same'
            else null end as mark
FROM   cte
ORDER BY 1, 3, 4, 2;
```

###### Задание 7

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— ранг эпизода с пропусками по убыванию среднего рейтинга (назовите поле ***rnk***);

Результат отсортируйте по возрастанию ***serial_name***, ***season***, ***episode***, ***episode_name***.

``` PLSQL
with cte as (SELECT t.original_title as serial_name,
                    (SELECT t.original_title
              FROM   titles as t
              WHERE  t.id = e.title_id) as episode_name, e.parent_id, e.season, e.episode, r.avg_rating
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id join titles as t
                     ON t.id = e.parent_id)
SELECT serial_name,
       episode_name,
       season,
       episode,
       rank() OVER (PARTITION BY parent_id
                    ORDER BY avg_rating desc) as rnk
FROM   cte
ORDER BY 1, 3, 4, 2;
```

###### Задание 8

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— рейтинг этого эпизода;

— максимальный рейтинг эпизодов среди последних 3 эпизодов, включая текущий (назовите поле ***max_rating***);

Результат отсортируйте по возрастанию ***serial_name***, ***season***, ***episode***, ***episode_name***.

``` PLSQL
with cte as (SELECT t.original_title as serial_name,
                    (SELECT t.original_title
              FROM   titles as t
              WHERE  t.id = e.title_id) as episode_name, e.parent_id, e.season, e.episode, r.avg_rating
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id join titles as t
                     ON t.id = e.parent_id)
SELECT serial_name,
       episode_name,
       season,
       episode,
       avg_rating,
       max(avg_rating) OVER (PARTITION BY parent_id
                             ORDER BY season, episode rows between 2 preceding and current row) as max_rating
FROM   cte
ORDER BY 1, 3, 4, 2;
```

###### Задание 9

Напишите SQL-запрос, который выведет количество умерших людей из таблицы ***persons*** для каждого года, начиная с 2000-го, накопленным итогом для текущего года и всех последующих лет.

Например, для 2012 года будет указано количество людей, которые ушли из жизни с 2012 по 2024 год.

Поля к выводу: год (***year_of_death***), кумулятивная сумма на этот год (***cume_cnt***).

Результат отсортируйте по возрастанию ***year_of_death***, ***cume_cnt***.

``` PLSQL
with cte as (SELECT year_of_death,
                    count(1) as cnt
             FROM   persons
             WHERE  year_of_death >= 2000
             GROUP BY year_of_death)
SELECT year_of_death,
       sum(cnt) OVER (ORDER BY year_of_death rows between current row and unbounded following) as cume_cnt
FROM   cte
ORDER BY 1, 2;
```

###### Задание 10

Напишите SQL-запрос, который выведет:

— название сериала (назовите поле ***serial_name***);

— название эпизода (назовите поле ***episode_name***);

— номер сезона;

— номер эпизода в сезоне;

— сквозной номер (назовите поле ***num***);

— максимальный рейтинг эпизодов до этой серии включительно (назовите поле ***max_rating***);

— количество голосов, отданных за сериал до этой серии включительно (назовите поле ***vote_cnt***);

``` PLSQL
with cte as (SELECT t.original_title as serial_name,
                    (SELECT t.original_title
              FROM   titles as t
              WHERE  t.id = e.title_id) as episode_name, e.parent_id, e.season, e.episode, r.avg_rating, r.vote_cnt
             FROM   episodes as e join ratings as r
                     ON e.title_id = r.title_id join titles as t
                     ON t.id = e.parent_id)
SELECT serial_name,
       episode_name,
       season,
       episode,
       row_number() OVER w as num,
       max(avg_rating) OVER w as max_rating,
       sum(vote_cnt) OVER w as vote_cnt
FROM   cte window w as (
PARTITION BY parent_id
ORDER BY season, episode)
```
